## P0 Test Suite
#### Original Author: Emil Sekerinski, McMaster University, February 2017
#### Modified by Dom DiPasquale, Patrick Jakubowski, and Mustafa Haddara

In [5]:
import os
import nbimporter
from P0 import compileString
from ST import printSymTab

The following method compiles the provided source both with and without optimizations and renders the results nicely for comparison.

In [6]:
def compareOutput(src):
    compileString(src, 'reg', target='mips')
    compileString(src, 'opt', target='opt')
    
    with open('reg') as f, open('opt') as o:
        reg = ['Orginal Compiler'] + f.read().split('\n')
        opt = ['Optmized Compiler'] + o.read().split('\n')
    os.remove('reg')
    os.remove('opt')
    
    tabsize = 8
    def rmtabs(s):
        res = ''
        for i in range(len(s)):
            if s[i] == '\t':
                res = res + ' ' * (tabsize-(i%tabsize))
            else:
                res = res + s[i]
        return res

    maxlinelen = 30
    def rightpad(s, n=maxlinelen, c=' '):
        clean = rmtabs(s)
        while len(clean) < n:
            clean = clean + c
        return clean
    
    for i in range(len(reg)):
        ls = rightpad(reg[i]) + '| '
        rs = opt[i] if i < len(opt) else ''
        print(ls+rs)

In [7]:
compareOutput("""program p;
      type T = record d,e,f:integer end;
      type A = array[0..9] of integer;
      var c: integer;
      var b: boolean;
      var w: A;
      procedure q(var z: integer);
            type T = record d,e:integer end;
            type A = array[0..9] of integer;
            var y: integer;
            var m,n: boolean;
            var w: T;
            var wd, we: integer;

            begin b:=true; w.d := 9; w.e := 5; w.d := 3 - 1 end;
      begin 
      w[1] := 9; w[2] := 5;
        c := 10;
        c := c + 1;
        write(c);
        q(c)
      end
    """)

TODO
Orginal Compiler              | Optmized Compiler
        .data                 | 	.data
w_:     .space 40             | 	.text
b_:     .space 4              | 	.globl q
c_:     .space 4              | 	.ent q
        .text                 | q:	
        .globl q              | 	sub $fp, $sp, 4
        .ent q                | 	sub $sp, $fp, 16
q:                            | 	addi $t6, $0, 1
        sw $fp, -8($sp)       | 	addi $t3, $0, 9
        sw $ra, -12($sp)      | 	addi $t0, $0, 5
        sub $fp, $sp, 4       | 	addi $t2, $0, 2
        sub $sp, $fp, 36      | 	add $sp, $fp, 4
        addi $t6, $0, 1       | 	lw $ra, -8($fp)
        sw $t6, b_            | 	lw $fp, -4($fp)
        addi $t3, $0, 9       | 	jr $ra
        sw $t3, -28($fp)      | 	.globl main
        addi $t0, $0, 5       | 	.ent main
        sw $t0, -24($fp)      | main:	
        addi $t2, $0, 2       | 	addi $t8, $0, 9
        sw $t2, -28($fp)      | 	addi $t1, $0, 5
        add $sp, $fp, 4       | 	addi $t4,

In [ ]:
compareOutput("""program p;
  var g: integer;          {global variable}
  procedure q(v: integer); {value parameter}
    var l: integer;        {local variable}
    procedure qx(v: integer); {value parameter}
      var l: integer;        {local variable}
      procedure qxt(v: integer); {value parameter}
        var l: integer;        {local variable}
        begin
          l := 9
        end;
      begin
        l := 9;
        q(l)
      end;
    procedure qc(v: integer); {value parameter}
      var l: integer;        {local variable}
      begin
        l := 9;
        q(l)
      end;
    begin
      l := 9;
      qx(l);
      if l > v then
         write(l)
      else
         write(g)
    end;
  procedure x(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      q(l)
    end;
  begin
    g := 5;
    q(7)
  end
""")

In [ ]:
compareOutput("""program p;
  var g: integer;          {global variable}
  procedure x(v: integer); {value parameter}
    var l: integer;        {local variable}
    procedure x2(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      x2(l)
    end;
    begin
      l := 9;
      x2(l)
    end;
  begin
    g := 5;
    x(g)
  end
""")